#Boiler Plate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

#Load Data

All Data can be found at this anonymous repo https://anonymous.4open.science/r/CHILData-4164/

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/trainingPerfusionCloseClin.pkl", "rb")
trainingPerfusion = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/validationPerfusionCloseClin.pkl", "rb")
validationPerfusion = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/testPerfusionCloseClin.pkl", "rb")
testPerfusion = pickle.load(o)

o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/trainingClinicalCloseClin.pkl", "rb")
trainingClinical = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/validationClinicalCloseClin.pkl", "rb")
validationClinical = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/testClinicalCloseClin.pkl", "rb")
testClinical = pickle.load(o)


In [ ]:
labelTraining = np.append(np.ones(54), np.zeros(54))
labelValTest = np.append(np.ones(18), np.zeros(18))

# Compare Methods

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
def bootstrapAUC(true, preds, bootstrap = 5000):
  AUCTotal = np.zeros([0])
  if((true.sum()<1) or (true.shape[0]-true.sum() <1)):
    print("Less than 1 true or false")
    return AUCTotal
  for x in range(bootstrap):
    np.random.seed(x)
    randInt = np.random.randint(0,true.shape[0], true.shape[0])
    trueSub = true[randInt]
    predSub = preds[randInt]

    while((trueSub.sum()<1) or (trueSub.shape[0]-trueSub.sum() <1)):
      randInt = np.random.randint(0,true.shape[0], true.shape[0])
      trueSub = true[randInt]
      predSub = preds[randInt]

    AUC = roc_auc_score(trueSub, predSub)
    AUCTotal = np.append(AUCTotal, AUC)

  return AUCTotal

#Logistic Regression

In [ ]:
def getAllPredsLR(modelFunc, perf, clin, testPerf, testClin):
  samplePredictions = np.zeros([0])
  sampleLabels = np.zeros([0])


  for j in range(12):
    for k in range(5):
      print("{}-{}".format(j,k))
      #Create Dataset
      X = np.append(perf[j,k].reshape(108,-1), clin[j,k], axis = 1) #The first column is the label
      Y = labelTraining

      #Create TEST
      TEST = np.append(testPerf[j,k].reshape(36,-1), testClin[j,k], axis = 1) #The first column is the label

      model = modelFunc(random_state = 0, max_iter = 10000)
      model.fit(X,Y)
      print("Training: " + str(model.score(X,Y)) + " Test: " + str(model.score(TEST,labelValTest)))
      prediction = model.predict_proba(TEST)[:,1]
      samplePredictions = np.append(samplePredictions, prediction)
      sampleLabels = np.append(sampleLabels, testClin[j,k])

  
  return samplePredictions, sampleLabels



In [ ]:
LRPreds, LRLabel = getAllPredsLR(LogisticRegression, trainingPerfusion, trainingClinical,testPerfusion, testClinical )

In [ ]:
#Save the generated predictions for later analysis
f = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/LRPredsCloseClin.pkl","wb")
pickle.dump(LRPreds,f)
f.close()

In [ ]:
#Open the saved predictions
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/LRPredsCloseClin.pkl", "rb")
LRPreds = pickle.load(o)
LRLabel = np.tile(np.append(np.ones([18]), np.zeros([18])), 60)

In [ ]:
LRAUCTotal = bootstrapAUC(LRLabel, LRPreds)

In [ ]:
LRAUCTotal.mean()
#00.80103312655945

0.8015514124671221

In [ ]:
print("[{:.4f} - {:.4f}]".format(np.percentile(LRAUCTotal, 2.5),np.percentile(LRAUCTotal, 97.5)))
#[0.7807 - 0.8206]

[0.7813 - 0.8213]


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def getAllPredsDT(perf, clin, testPerf, testClin):
  samplePredictions = np.zeros([0])
  sampleLabels = np.zeros([0])


  for j in range(12):
    for k in range(5):
      print("{}-{}".format(j,k))
      #Create Dataset
      X = np.append(perf[j,k].reshape(108,-1), clin[j,k], axis = 1) 
      Y = labelTraining

      #Create TEST
      TEST = np.append(testPerf[j,k].reshape(36,-1), testClin[j,k], axis = 1) #The first column is the label

      model = DecisionTreeClassifier(random_state = 0)
      model.fit(X,Y)
      print("Training: " + str(model.score(X,Y)) + " Test: " + str(model.score(TEST,labelValTest)))
      prediction = model.predict_proba(TEST)[:,1]
      samplePredictions = np.append(samplePredictions, prediction)
      sampleLabels = np.append(sampleLabels, labelValTest)

  
  return samplePredictions, sampleLabels



In [ ]:
DTPreds, DTLabel = getAllPredsDT(trainingPerfusion, trainingClinical,testPerfusion, testClinical )

In [ ]:
f = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/DTPredsCloseClin.pkl","wb")
pickle.dump(DTPreds,f)
f.close()

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/DTPredsCloseClin.pkl", "rb")
DTPreds = pickle.load(o)

In [ ]:
DTAUCTotal = bootstrapAUC(LRLabel, DTPreds)

In [ ]:
DTAUCTotal.mean()
#0.7577265261291511

0.7577265261291511

In [ ]:
print("[{:.4f} - {:.4f}]".format(np.percentile(DTAUCTotal, 2.5),np.percentile(DTAUCTotal, 97.5)))
#[0.7410 - 0.7741]

[0.7410 - 0.7741]


# Native Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
def getAllPredsNB(perf, clin, testPerf, testClin):
  samplePredictions = np.zeros([0])
  sampleLabels = np.zeros([0])


  for j in range(12):
    for k in range(5):
      print("{}-{}".format(j,k))
      #Create Dataset
      X = np.append(perf[j,k].reshape(108,-1), clin[j,k], axis = 1) 
      Y = labelTraining

      #Create TEST
      TEST = np.append(testPerf[j,k].reshape(36,-1), testClin[j,k], axis = 1) 

      model = GaussianNB()
      model.fit(X,Y)
      print("Training: " + str(model.score(X,Y)) + " Test: " + str(model.score(TEST,labelValTest)))
      prediction = model.predict_proba(TEST)[:,1]
      samplePredictions = np.append(samplePredictions, prediction)
      sampleLabels = np.append(sampleLabels, labelValTest)

  
  return samplePredictions, sampleLabels



In [ ]:
NBPreds, NBLabel = getAllPredsNB(trainingPerfusion, trainingClinical,testPerfusion, testClinical)

In [ ]:
f = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/NBPredsCloseClin.pkl","wb")
pickle.dump(NBPreds,f)
f.close()

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/NBPredsCloseClin.pkl", "rb")
NBPreds = pickle.load(o)

In [ ]:
NBAUCTotal = bootstrapAUC(LRLabel, NBPreds)

In [ ]:
NBAUCTotal.mean()
#0.770665747161304

0.770665747161304

In [ ]:
print("[{:.4f} - {:.4f}]".format(np.percentile(NBAUCTotal, 2.5),np.percentile(NBAUCTotal, 97.5)))
#[0.7537 - 0.7873]

[0.7537 - 0.7873]


# MLPClassifier

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
!pip install keras-tuner
import kerastuner as kt
from kerastuner import HyperParameters as hp

In [ ]:
def returnModel(hp):
  model = tf.keras.Sequential()
  units = hp.Int("units", 1,16435, 64)
  model.add(tf.keras.layers.Dense(units, activation = "selu", input_shape=(16435,))) #16464
  model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
  opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(optimizer=opt, loss='binary_crossentropy')
  return model

In [ ]:
tuner = kt.BayesianOptimization(
  returnModel,                   
  objective= "val_loss",
  max_trials=60,
  seed =0, 
  overwrite=True,
  beta = 100          
)
XTrain = np.append(trainingPerfusion[0,0].reshape(108,-1), trainingClinical[0,0], axis = 1) 
YTrain = labelTraining

XVal = np.append(validationPerfusion[0,0].reshape(36,-1), validationClinical[0,0], axis = 1)
YVal = labelValTest

es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=20, min_delta = 0.001)
tuner.search(                                                           
    XTrain,
    YTrain,
    epochs=10000,                                                         
    validation_data = (XVal, YVal),
    callbacks=[es],
    verbose=1,
)
print(tuner.get_best_hyperparameters(1)[0].values)

In [ ]:
def returnTrueModel():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = "selu", input_shape=(16435,))) 
  model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
  opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(optimizer=opt, loss='binary_crossentropy')
  return model

In [ ]:
def getAllPredsMLP(modelFunc, perf, clin, valPerf, valClin, testPerf, testClin):
  samplePredictions = np.zeros([0])
  sampleLabels = np.zeros([0])


  for j in range(12):
    for k in range(5):
      print("{}-{}".format(j,k))
      #Create Dataset
      XTrain = np.append(perf[j,k].reshape(108,-1), clin[j,k], axis = 1) 
      YTrain = labelTraining

      XVal = np.append(valPerf[j,k].reshape(36,-1), valClin[j,k], axis = 1) 
      YVal = labelValTest

      #Create TEST
      TEST = np.append(testPerf[j,k].reshape(36,-1), testClin[j,k], axis = 1)
      es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=20, min_delta = 0.001)
      mc = ModelCheckpoint("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/DENSE/CloseClinModelK{}Fold{}.h5".format(j, k), monitor='val_loss', mode='min', verbose=0, save_best_only=True)

      model = modelFunc()
      model.fit(
          XTrain,
          YTrain,
          epochs = 10000,
          validation_data = (XVal,YVal),
          verbose = 1,
          callbacks=[es, mc]
          )
      
      prediction = model.predict(TEST)
      samplePredictions = np.append(samplePredictions, prediction)
      sampleLabels = np.append(sampleLabels, labelValTest)

  
  return samplePredictions, sampleLabels



In [ ]:
MLPPreds, MLPLabel = getAllPredsMLP(returnTrueModel,trainingPerfusion, trainingClinical,validationPerfusion, validationClinical,testPerfusion, testClinical )

In [ ]:
f = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/MLPPredsCloseClin.pkl","wb")
pickle.dump(MLPPreds,f)
f.close()

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/MLPPredsCloseClin.pkl", "rb")
MLPPreds = pickle.load(o)

In [ ]:
MLPAUCTotal = bootstrapAUC(LRLabel, MLPPreds)

In [ ]:
MLPAUCTotal.mean()
#0.7299333876487715

0.7299333876487715

In [ ]:
print("[{:.4f} - {:.4f}]".format(np.percentile(MLPAUCTotal, 2.5),np.percentile(MLPAUCTotal, 97.5)))
#[0.7080 - 0.7513]

[0.7080 - 0.7513]


#Comparison with L = 100

In [ ]:
def bootstrapComaprisonAUC(trueOne, predsOne,trueTwo, predsTwo, bootstrap = 5000):
  diffAUCTotal = np.zeros([0])

  if((trueOne.sum()<1) or (trueOne.shape[0]-trueOne.sum() <1)):
    print("Less than 1 true or false")
    return diffAUCTotal
  if((trueTwo.sum()<1) or (trueTwo.shape[0]-trueTwo.sum() <1)):
    print("Less than 1 true or false")
    return diffAUCTotal

  intialTestStat = roc_auc_score(trueOne, predsOne) - roc_auc_score(trueTwo, predsTwo)
  lengthOne = trueOne.shape[0]
  lengthTwo = trueTwo.shape[0]
  totalTrueGroup = np.append(trueOne, trueTwo)
  totalPredGroup = np.append(predsOne, predsTwo)
  
  for x in range(bootstrap):
    np.random.seed(x)
    randInt = np.random.randint(0,lengthOne+lengthTwo, lengthOne)
    leftOvers = np.setdiff1d(np.arange(lengthOne+lengthTwo), randInt)
    trueOneSub = totalTrueGroup[randInt]
    predOneSub = totalPredGroup[randInt]
    trueTwoSub = totalTrueGroup[leftOvers]
    predTwoSub = totalPredGroup[leftOvers]

    while((trueOneSub.sum()<1) or (trueOneSub.shape[0]-trueOneSub.sum() <1) or (trueTwoSub.sum()<1) or (trueTwoSub.shape[0]-trueTwoSub.sum() <1)):
      randInt = np.random.randint(0,lengthOne+lengthTwo, lengthOne)
      leftOvers = np.setdiff1d(np.arange(lengthOne+lengthTwo), randInt)
      trueOneSub = totalTrueGroup[randInt]
      predOneSub = totalPredGroup[randInt]
      trueTwoSub = totalTrueGroup[leftOvers]
      predTwoSub = totalPredGroup[leftOvers]

    AUCOne = roc_auc_score(trueOneSub, predOneSub)
    AUCTwo = roc_auc_score(trueTwoSub, predTwoSub)
    diff = AUCOne - AUCTwo
    diffAUCTotal = np.append(diffAUCTotal, diff)
  
  pValue = (diffAUCTotal>intialTestStat).sum()/diffAUCTotal.shape[0]

  return pValue

In [ ]:
#Open up prediction from the models trained on the L=1 GPR
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/predictionsCloseClin", "rb")
predictions = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/predictionsLargeCloseClin.pkl", "rb")
predictionsLarge = pickle.load(o)

o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/LRPredsLargeCloseClin.pkl", "rb")
LRPredsLarge = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/MLPPredsLargeCloseClin.pkl", "rb")
MLPPredsLarge = pickle.load(o)

In [ ]:
LSTMAUCDiff = bootstrapComaprisonAUC(LRLabel, predictions.flatten(),LRLabel, predictionsLarge.flatten() , 5000)
LRAUCDiff = bootstrapComaprisonAUC(LRLabel, LRPreds.flatten(),LRLabel, LRPredsLarge.flatten(), 5000)
MLPAUCDiff = bootstrapComaprisonAUC(LRLabel, MLPPreds.flatten(),LRLabel, MLPPredsLarge.flatten(), 5000)

In [ ]:
print(LSTMAUCDiff)
print(LRAUCDiff)
print(MLPAUCDiff)

0.0
0.4626
0.17
